In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui
import dbManagement as dbm

In [3]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [86]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [4]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [99]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL020/2023-05-04/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
frame2time had 2 more than suite2p output. This happens sometimes. I don't like it. I think it's because scanimage sends a TTL before starting the frame
Computing red cell features for ATL020/2023-05-04/702... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
C:\Users\andrew\Documents\localData\ATL020\2023-05-04\702


In [5]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [107]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL027','2023-08-10','701')

\\zaru.cortexlab.net\Subjects\ATL027\2023-08-10\701
C:\Users\andrew\Documents\localData\ATL027\2023-08-10\701


In [13]:
print(vrexp.sessionPrint())
latestModification = 0
for p in vrexp.suite2pPath().rglob("*"):
    latestModification = max(p.stat().st_mtime, latestModification)

import datetime 
datetime.datetime.fromtimestamp(latestModification)

ATL020/2023-05-04/702


datetime.datetime(2023, 8, 14, 12, 40, 17, 910041)

In [35]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------

In [2]:
vrdb = dbm.vrDatabase()

In [3]:
vrdb.tableData()[0]

['uSessionID',
 'mouseName',
 'sessionDate',
 'sessionID',
 'experimentType',
 'experimentID',
 'variableGain',
 'behavior',
 'imaging',
 'faceCamera',
 'vrEnvironments',
 'headPlateRotation',
 'numPlanes',
 'planeSeparation',
 'pockelsPercentage',
 'objectiveRotation',
 'vrRegistration',
 'suite2p',
 'suite2pQC',
 'redCellQC',
 'sessionQC',
 'scratchJustification',
 'logtime',
 'sessionNotes',
 'suite2pDate',
 'suite2pTime',
 'vrRegistrationDate',
 'vrRegistrationTime']

In [7]:
df = vrdb.showTable()

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [5]:
vrdb.updateSuite2pDateTime()

In [ ]:
vrdb.registerSessions()

In registerSessions, 'vrBehaviorVersion' is an important input that hasn't been coded yet!

Performing vrExperiment preprocessing for session: ATL012/2023-01-30/702
In session: ATL012/2023-01-30/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
The following npy files are present in some but not all plane folders within session ATL012/2023-01-30/702: ['spks', 'stat', 'Fneu', 'Fneu_chan2', 'redcell', 'iscell', 'F', 'F_chan2']
Each plane folder contains the following npy files: ['ops']
The following exception was raised when trying to preprocess session: ATL012/2023-01-30/702
Exception: ATL012/2023-01-30/702 is missing stat in at least one suite2p folder!

Performing vrExperiment preprocessing for session: ATL012/2023-02-17/701
The following exception was raised when trying to preprocess session: ATL012/2023-02